**Disclaimer:** Se utilizó ChatGPT para el Reducer de los cuadrados, aunque están basados en el reducer del triangulo que es más simple.

In [8]:
!pip install pyspark

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

24/06/15 20:21:20 WARN Utils: Your hostname, cristobal-IdeaPad-3-15ALC6 resolves to a loopback address: 127.0.1.1; using 192.168.1.160 instead (on interface wlp1s0)
24/06/15 20:21:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/15 20:21:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=pyspark-shell>

In [10]:
data = [
    (1,11,2),(1,11,3),(2,11,3),
    (3,11,2),(3,11,4),(4,11,1),
    (4,11,2),(4,11,3),(4,12,5),
    (5,12,1),(5,12,2),(5,12,6)
    ]

**Parte 1**

1.1 Precalentamiento: Triángulos.



**Fase de Map.**

Cada mapper tendra una cierta cantidad de aristas del grafo. Para cada arista (n1, R, n2),
el mapper debe hacer lo siguiente.

1. Hashear los nodos: b1 = h(n1), b2 = h(n2).
2. Como el triangulo que formemos solo puede mapear la variable x a n1 e y a n2, la arista (n1, R, n2)
solo podrıa formar parte de un trıangulo de la forma (n1, n2, a) para un a ∈V . Entonces, generamos b
llaves (b1, b2, 0), . . . , (b1, b2, b −1), y emitimos b mensajes con el contenido de la arista, uno para cada
llave: ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))

Para la primera parte, realizaremos un una función que permita hashear de la forma (key, arista)

**Probamos para Triangulos (X,11,Y), (Y,11,Z), (Z,11,X)**

In [11]:
def key_edge(edge, b):
  map = [] # Lista que contiene los pares key, arista para una determinada edge
  n1, label, n2 = edge # Descomponemos la arista entregada
  for i in range (b): #Iteramos en los valores de mod 2: 0,1
  #agregamos los pares ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))
    map.append(((n1%b, n2%b, i), edge)) #(x,11,y)
    map.append(((i, n1%b, n2%b), edge))  # (y, 11, z)
    map.append(((n2%b, i, n1%b), edge))  # (z, 11, x)
  return map



In [12]:
data_triangles = [edge for edge in data if edge[1] == 11]

rdd = sc.parallelize(data_triangles)
rdd.collect()


[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3)]

In [13]:
map_rdd = rdd.flatMap(lambda x: key_edge(x, 2))
map_rdd.collect()

[((1, 0, 0), (1, 11, 2)),
 ((0, 1, 0), (1, 11, 2)),
 ((0, 0, 1), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 2)),
 ((0, 1, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 3)),
 ((0, 1, 1), (1, 11, 3)),
 ((1, 0, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((0, 0, 1), (2, 11, 3)),
 ((1, 0, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((1, 0, 1), (2, 11, 3)),
 ((1, 1, 0), (2, 11, 3)),
 ((1, 0, 0), (3, 11, 2)),
 ((0, 1, 0), (3, 11, 2)),
 ((0, 0, 1), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((1, 1, 0), (3, 11, 2)),
 ((0, 1, 1), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 4)),
 ((0, 1, 0), (3, 11, 4)),
 ((0, 0, 1), (3, 11, 4)),
 ((1, 0, 1), (3, 11, 4)),
 ((1, 1, 0), (3, 11, 4)),
 ((0, 1, 1), (3, 11, 4)),
 ((0, 1, 0), (4, 11, 1)),
 ((0, 0, 1), (4, 11, 1)),
 ((1, 0, 0), (4, 11, 1)),
 ((0, 1, 1), (4, 11, 1)),
 ((1, 0, 1), (4, 11, 1)),
 ((1, 1, 0), (4, 11, 1)),
 ((0, 0, 0), (4, 11, 2)),
 ((0, 0, 0), (4, 11, 2)),
 ((0, 0, 0),

**Fase de Reduce.**

 Los reducers reciben siempre los mensajes correspondientes a alguna llave (b1, b2, b3).
Pero en el valor de esa llave se encuentran aristas. Todas estas aristas forman un pequeño grafo , y el reducer
emite como respuesta todas las tuplas (n1, n2, n3) correspondientes a los triangulos que detecta en su grafo

In [14]:
group_rdd = map_rdd.groupByKey().mapValues(list)
group_rdd.collect()

[((1, 0, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((0, 1, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((1, 0, 1),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((0, 0, 1),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((1, 1, 0),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((0, 0, 0), [(4, 11, 2), (4, 11, 2), (4, 11, 2)]),
 ((0, 1, 1),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((1, 1, 1), [(1, 11, 3), (1, 11, 3), (1, 11, 3)])]

**Reducer Triangulos**

In [15]:
def reduce_phase(values):
    from collections import defaultdict
    edges_by_node = defaultdict(list)
    for n1, R, n2 in values:
        edges_by_node[(n1, n2)].append(R)

    # Find triangles
    triangles = set()
    nodes = list(edges_by_node.keys())
    for i in range(len(nodes)):
        for j in range(i+1, len(nodes)):
            for k in range(j+1, len(nodes)):
                n1, n2 = nodes[i]
                n2_2, n3 = nodes[j]
                n3_2, n1_2 = nodes[k]
                if n2 == n2_2 and n3 == n3_2 and n1 == n1_2:
                    triangles.add((n1, n2, n3))
    return list(triangles)


In [16]:
rdd_triangles = group_rdd.flatMapValues(reduce_phase)
rdd_triangles.collect()

[((1, 0, 0), (2, 3, 4)),
 ((0, 1, 0), (2, 3, 4)),
 ((1, 0, 1), (1, 3, 4)),
 ((0, 0, 1), (2, 3, 4)),
 ((1, 1, 0), (1, 3, 4)),
 ((0, 1, 1), (1, 3, 4))]

**Consulta Cuadrado (x,11,y), (y,11,z), (z,11,w), (w,11,x)**

In [17]:
def key_edge_square(edge, b):
  map = [] # Lista que contiene los pares key, arista para una determinada edge
  n1, label, n2 = edge # Descomponemos la arista entregada
  for i in range (b): #Iteramos en los valores de mod
  #agregamos los pares ((b1, b2, 0), (n1, R, n2)), . . . , ((b1, b2, b −1), (n1, R, n2))
    map.append(((n1%b, n2%b, i), edge)) #(x,11,y)
    map.append(((i, n1%b, n2%b), edge))  # (y, 11, z)
    map.append(((n2%b, i, n1%b), edge))  # (z, 11, w)
    map.append(((n1%b, n2%b, i), edge))  # (w, 11, x)
  return map


In [18]:
rdd = sc.parallelize(data_triangles)
rdd.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3)]

In [19]:
map_rdd_square = rdd.flatMap(lambda x: key_edge_square(x, 2))
map_rdd_square.collect()

[((1, 0, 0), (1, 11, 2)),
 ((0, 1, 0), (1, 11, 2)),
 ((0, 0, 1), (1, 11, 2)),
 ((1, 0, 0), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 2)),
 ((0, 1, 1), (1, 11, 2)),
 ((1, 0, 1), (1, 11, 2)),
 ((1, 1, 0), (1, 11, 3)),
 ((0, 1, 1), (1, 11, 3)),
 ((1, 0, 1), (1, 11, 3)),
 ((1, 1, 0), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((1, 1, 1), (1, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((0, 0, 1), (2, 11, 3)),
 ((1, 0, 0), (2, 11, 3)),
 ((0, 1, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((1, 0, 1), (2, 11, 3)),
 ((1, 1, 0), (2, 11, 3)),
 ((0, 1, 1), (2, 11, 3)),
 ((1, 0, 0), (3, 11, 2)),
 ((0, 1, 0), (3, 11, 2)),
 ((0, 0, 1), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((1, 1, 0), (3, 11, 2)),
 ((0, 1, 1), (3, 11, 2)),
 ((1, 0, 1), (3, 11, 2)),
 ((1, 0, 0), (3, 11, 4)),
 ((0, 1, 0), (3, 11, 4)),
 ((0, 0, 1), (3, 11, 4)),
 ((1, 0, 0), (3, 11, 4)),
 ((1, 0, 1), (3, 11, 4)),
 ((1, 1, 0), (3, 11, 4)),
 ((0, 1, 1),

In [20]:
rdd_group= map_rdd_square.groupByKey().mapValues(list)
rdd_group.collect()


[((1, 0, 0),
  [(1, 11, 2),
   (1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3)]),
 ((0, 1, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3),
   (4, 11, 3)]),
 ((1, 0, 1),
  [(1, 11, 2),
   (1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((0, 0, 1),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 2),
   (4, 11, 3)]),
 ((1, 1, 0),
  [(1, 11, 2),
   (1, 11, 3),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3)]),
 ((0, 0, 0), [(4, 11, 2), (4, 11, 2), (4, 11, 2), (4, 11, 2)]),
 ((0, 1, 1),
  [(1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 1),
   (4, 11, 3),
   (4, 11, 3)]),
 ((1, 

In [21]:
def reduce_phase_square_same_label(values):
    from collections import defaultdict

    out_edges = defaultdict(list)

    for n1, R, n2 in values:
        if R == 11:  # We only consider edges with label 11 for this query
            out_edges[n1].append(n2)

    squares = set()
    for n1 in out_edges:
        for n2 in out_edges[n1]:
            if n2 in out_edges:
                for n3 in out_edges[n2]:
                    if n3 in out_edges:
                        for n4 in out_edges[n3]:
                            if n4 in out_edges and n1 in out_edges[n4]:
                                if len(set([n1, n2, n3, n4])) == 4:
                                    square = tuple(sorted([n1, n2, n3, n4]))
                                    squares.add(square)
    return list(squares)





In [22]:
rdd_squares = rdd_group.flatMapValues(reduce_phase_square_same_label)
rdd_squares.collect()

[((1, 0, 0), (1, 2, 3, 4)),
 ((0, 1, 0), (1, 2, 3, 4)),
 ((1, 0, 1), (1, 2, 3, 4)),
 ((0, 0, 1), (1, 2, 3, 4)),
 ((1, 1, 0), (1, 2, 3, 4)),
 ((0, 1, 1), (1, 2, 3, 4))]

**Consulta Cuadrado: (x,11,y), (y,11,z), (z,12,w), (w,12,x)**

In [23]:
rdd = sc.parallelize(data)
rdd.collect()

[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6)]

In [24]:
rdd_map = rdd.flatMap(lambda x: key_edge_square(x, 2))
rdd_grouped = rdd_map.groupByKey().mapValues(list)
rdd_grouped.collect()

[((1, 0, 0),
  [(1, 11, 2),
   (1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3),
   (4, 12, 5),
   (5, 12, 2),
   (5, 12, 2),
   (5, 12, 6),
   (5, 12, 6)]),
 ((0, 1, 0),
  [(1, 11, 2),
   (2, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 3),
   (4, 11, 3),
   (4, 12, 5),
   (4, 12, 5),
   (5, 12, 2),
   (5, 12, 6)]),
 ((1, 0, 1),
  [(1, 11, 2),
   (1, 11, 2),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 2),
   (3, 11, 4),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 3),
   (4, 12, 5),
   (5, 12, 1),
   (5, 12, 2),
   (5, 12, 2),
   (5, 12, 6),
   (5, 12, 6)]),
 ((0, 0, 1),
  [(1, 11, 2),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4),
   (4, 11, 1),
   (4, 11, 2),
   (4, 11, 2),
   (4, 11, 3),
   (4, 12, 5),
   (5, 12, 2),
   (5, 12, 6)]),
 ((1, 1, 0),
  [(1, 11, 2),
   (1, 11, 3),
   (1, 11, 3),
   (2, 11, 3),
   (3, 11, 2),
   (3, 11, 4

In [38]:
def reduce_phase_square_mixed_labels(values):
    from collections import defaultdict

    out_edges = defaultdict(lambda: defaultdict(list))

    for n1, R, n2 in values:
        out_edges[n1][R].append(n2)

    squares = set()
    for n1 in out_edges:
        for n2 in out_edges[n1][11]:  # Edge (n1, 11, n2)
            if n2 in out_edges:
                for n3 in out_edges[n2][11]:  # Edge (n2, 11, n3)
                    if n3 in out_edges:
                        for n4 in out_edges[n3][12]:  # Edge (n3, 12, n4)
                            if n4 in out_edges and n1 in out_edges[n4][12]:  # Edge (n4, 12, n1)
                                if len(set([n1, n2, n3, n4])) == 4:
                                    square = tuple(sorted([n1, n2, n3, n4]))
                                    squares.add(square)
    return list(squares)



In [26]:
rdd_squares_mixed_labels = rdd_grouped.flatMapValues(reduce_phase_square_mixed_labels)
for l in rdd_squares_mixed_labels.collect():
    print(l[1])

(2, 3, 4, 5)
(2, 3, 4, 5)
(1, 3, 4, 5)
(2, 3, 4, 5)
(2, 3, 4, 5)
(1, 3, 4, 5)
(2, 3, 4, 5)
(1, 3, 4, 5)
(2, 3, 4, 5)


# Parte "3"
Asume ahora que recibes un subgrafo como tres arreglos: un arreglo A con las variables, otro L con los
tipos de aristas, y una matriz M de tamaño |A| × |L| × |A| que tiene un uno en la posicion (x, R, y) si
y solo si (x, R, y) es una arista de tu subgrafo.

Implementa un programa en PySpark que reciba un patrón que tiene solo variables, y exactamente
cuatro variables, y entregue todos los matches de ese patrón (como tuplas de 4 nodos) en el grafo
usando b⁴ reducers, donde b es un parámetro.

In [32]:
A = ["w","x","y","z"]
L = [11,12]
     # 11       12        # Parte izq de la arista
M = [[[0,0,0,0],[1,0,0,0]],# w
    [[0,1,0,0],[0,0,0,0]], # x 
    [[0,0,1,0],[0,0,0,0]], # y
    [[0,0,0,0],[1,0,0,0]]]# z
# cada lista [0,0,0] indica si es w, x, y ó z
     
# En este ejemplo se ve un subgrafo que representa cuadrados (x,y,z) se veria asi-> (w,12,x), (x,11,y), (y,11,z), (z,12,x)

In [43]:
def funcion(A, M, b):
    assert len(A) == 4, f"El patrón {A} no tiene exactamente 4 variables"
    #Armar rdd
    data = [
    (1,11,2),(1,11,3),(2,11,3),
    (3,11,2),(3,11,4),(4,11,1),
    (4,11,2),(4,11,3),(4,12,5),
    (5,12,1),(5,12,2),(5,12,6)
    ]
    rdd = sc.parallelize(data)
    # Map
    map_rdd_square = rdd.flatMap(lambda x: key_edge_square(x, b))
    rdd_group= map_rdd_square.groupByKey().mapValues(list)

    # Reduce
    rdd_squares = rdd_group.flatMapValues(reduce_phase_square_mixed_labels) 
    #retornar
    lista = rdd_squares.collect()
    r = set()
    for l in lista:
        r.add(l[1])
    return r

In [46]:
A = ["w","x","y","z"]
L = [11,12]
     # 11       12        # Parte izq de la arista
M = [[[0,0,0,0],[1,0,0,0]],# w
    [[0,1,0,0],[0,0,0,0]], # x 
    [[0,0,1,0],[0,0,0,0]], # y
    [[0,0,0,0],[1,0,0,0]]]# z

funcion(A,M, 2) # A y M son el patrón, mientras que el 2 es el parámetro b

{(1, 3, 4, 5), (2, 3, 4, 5)}